In [1]:
!pip3 install transformers datasets sentencepiece peft -q
!pip install torch~=2.1.0 --index-url https://download.pytorch.org/whl/cpu -q # Updating torch since we need the latest version
!pip install torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html -q
!pip uninstall tensorflow -y # If we don't do this, TF may take over TPU and cause permission error for PT
!cp /kaggle/input/utils-xla/spmd_util.py . # Copy script to working directory from this repo https://github.com/HeegyuKim/torch-xla-SPMD


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.
torchaudio 2.0.0 requires torch==2.0.0, but you have torch 2.1.0+cpu which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboardx 2.6.1 requires protobuf>=4.22.3, but you h

In [2]:
import os
import pandas as pd
import numpy as np
import datasets
import torch.optim as optim
import torch_xla.debug.profiler as xp
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp # We also import mp modules if we wanna use that for some reason
import torch_xla.distributed.parallel_loader as pl
import torch_xla.test.test_utils as test_utils
import torch
import torch.nn as nn
import re
import torch_xla.experimental.xla_sharding as xs
import torch_xla.core.xla_model as xm
from transformers import (
    GPTNeoXConfig, T5Config, LlamaConfig, AutoTokenizer, AutoModelForCausalLM, DataCollatorWithPadding, AutoConfig, AutoModelForSequenceClassification
) # You can use any of models with those configs (even flan T5 xxl!). Other models are not supported.

from transformers import logging as hf_logging
import torch.nn.functional as F
import torch_xla.runtime as xr

xr.use_spmd()

import torch_xla.experimental.xla_sharding as xs # "experimental" prefix always means you're gonna have a good time LMAO
from torch_xla.experimental.xla_sharded_tensor import XLAShardedTensor
from torch_xla.experimental.xla_sharding import Mesh

from peft import LoraConfig, TaskType, get_peft_model # If we wanna use peft. Quantazation requiers GPU though. You'll have to download already quantazed models
from spmd_util import partition_module                # You could experiment with using already quantazed models like 4bit/Llama-2-7b-Chat-GPTQ if you're feeling funny
from datasets import Dataset, load_dataset, concatenate_datasets
from dataclasses import dataclass
from tqdm import tqdm

import transformers
import datasets
from datasets import Dataset
from sklearn.metrics import roc_auc_score

!export USE_TORCH=True #If we don't do this, transformers will seemingly bork the session upon import. Really weird error.
os.environ["PJRT_DEVICE"] = "TPU"
os.environ.pop('TPU_PROCESS_ADDRESSES')
os.environ.pop('CLOUD_TPU_TASK_ID')
hf_logging.set_verbosity_error() # It can still display warnings which is a bit annoying but whatever


MAX_INPUT=512
MODEL = "/kaggle/input/best-llm-starter-pack/nous_hermes/Nous-Hermes-Llama2-13b/snapshots/8f95aa9cd207db7b24179fc779c2b8973e71bee2" # Provide your own model directory

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is pandas processing done for Kaggle "Detect AI" competition. Provide your own dataset

In [3]:
df_ai = pd.read_csv('/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv')
ai_samples_500 = pd.read_csv('/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv')
persuade_df = pd.read_csv('/kaggle/input/persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv')
df_llama = pd.read_csv('/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_70b_v1.csv')
df_falcon = pd.read_csv('/kaggle/input/daigt-data-llama-70b-and-falcon180b/falcon_180b_v1.csv')
proper_train = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
aug_df = pd.read_csv('/kaggle/input/argugpt/argugpt.csv')
corrected_df = pd.read_csv('/kaggle/input/grammar-corrected-persuade-10k/corrected_persuade_10k.csv')

In [4]:
aug_df = pd.DataFrame({'text': aug_df['text'], 'generated': 1})
proper_train_ai = pd.DataFrame({'text': proper_train['text'][proper_train['label']==1], 'generated': 1}).reset_index(drop=True)
df_nbroad = pd.concat([df_llama, df_falcon]).reset_index(drop=True)
nbroad_ai = pd.DataFrame({'text': df_nbroad['generated_text'], 'generated': 1}).reset_index(drop=True)
ai_ds = pd.concat([proper_train_ai, nbroad_ai, ai_samples_500.drop(columns=['prompt_id', 'id']), aug_df]).reset_index(drop=True)
human_ds = pd.DataFrame({'text': corrected_df['corrected_text'], 'generated': 0}).reset_index(drop=True)
final_df = pd.concat([ai_ds, human_ds]).sample(frac=1).reset_index(drop=True)
final_df

lengths = []
for sample in final_df['text']:
    lengths.append(len(sample.split(' ')))
max(lengths), min(lengths), sum(lengths) / len(lengths) # Rough estimation of lengths of sentences

(1638, 4, 361.42858331946064)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token # Honestly like is pad token ever set to something else than EOS?

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=MAX_INPUT, padding='max_length', truncation=True) # We could try `padding_to_multiple_of=128`

In [7]:
ds = Dataset.from_pandas(final_df).rename_column("generated", "label").train_test_split(test_size=0.15)

ds['train'] = ds['train'].map(preprocess_function, batched=False, num_proc=96, remove_columns=['text']) # Kaggle TPU session's CPUs are copiously powerfull
ds['test'] = ds['test'].map(preprocess_function, batched=False, num_proc=96, remove_columns=['text'])
ds

Map (num_proc=96): 100%|██████████| 20423/20423 [00:28<00:00, 729.29 examples/s]
/usr/local/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Map (num_proc=96): 100%|██████████| 3605/3605 [00:25<00:00, 141.27 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 20423
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3605
    })
})

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2, torch_dtype=torch.bfloat16) # Some may argue training in pure bfloat16 is bad but it's fine for most purposes

Loading checkpoint shards: 100%|██████████| 3/3 [02:41<00:00, 53.85s/it]


In [9]:
FLAGS = {'MAX_INPUT': MAX_INPUT,
         'LOGGING_STEPS': 100,
         'NUM_EPOCHS': 1,
         'BATCH_SIZE': 8, #Making batch_size lower then 8 will result in slower training, but will allow for larger models\context. Fortunately, we have 128GBs. Setting higher batch_size doesn't seem to improve time.
          'NUM_STEPS': len(ds)} 

In [10]:
training_loader = torch.utils.data.DataLoader(ds['train'], batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer), shuffle=True) # We could reinvent the wheel by writing our own data collator
testing_loader = torch.utils.data.DataLoader(ds['test'], batch_size=FLAGS['BATCH_SIZE'], collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))

device = xm.xla_device()

In [11]:
cnt = 0
for param in model.parameters(): # Very direct way of freezing parameters
    cnt += 1
    param.requires_grad = False
    if cnt > 260:
        param.requires_grad = True

config = AutoConfig.from_pretrained(MODEL)
num_devices = xr.global_runtime_device_count()
mesh_shape = (1, num_devices, 1)
device_ids = np.array(range(num_devices))
mesh = Mesh(device_ids, mesh_shape, ('dp', 'fsdp', 'mp'))
partition_module(model, mesh) # After this, the model is sharded between TPU cores but still has the same API as if it was on single device. Neat.

In [12]:
!export XLA_USE_BF16=1 # Idk if this does anything
num_iterations = int(FLAGS['NUM_STEPS'] / FLAGS['BATCH_SIZE'])
lr = 1e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=FLAGS['NUM_STEPS'] * FLAGS['BATCH_SIZE'])

def training_loop(training_loader, epoch):
    model.train()
    print('Epoch {} train begin {}'.format(epoch, test_utils.now()))
    for step, batch in enumerate(training_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device),  torch.unsqueeze(batch.labels.to(device), 1) # Here we inhale some copium
        xs.mark_sharding(input_ids, mesh, (0, 1)) # Sharding inputs
        xs.mark_sharding(attention_mask, mesh, (0, 1))
        xs.mark_sharding(labels, mesh, (0, 1))
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        xm.mark_step()
        if step % FLAGS['LOGGING_STEPS'] == 0:
            print(f'loss: {loss.item()}, time: {test_utils.now()}, step: {step}')
        scheduler.step()
    print('Epoch {} train end {}'.format(epoch, test_utils.now()))

def testing_loop(testing_loader, epoch):
    model.eval()
    total_loss = 0.0
    total_steps = 0
    with torch.no_grad():
        for step, batch in enumerate(testing_loader):
            input_ids, attention_mask, labels = batch.input_ids.to(device), batch.attention_mask.to(device), torch.unsqueeze(batch.labels.to(device), 1) # Even more copium
            xs.mark_sharding(input_ids, mesh, (0, 1))
            xs.mark_sharding(attention_mask, mesh, (0, 1))
            xs.mark_sharding(labels, mesh, (0, 1))
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            total_steps += 1
    average_loss = total_loss / total_steps
    print('Epoch {} test end {}, test loss={:.2f}'.format(epoch, test_utils.now(), average_loss))

def train(FLAGS):
    for epoch in range(1, FLAGS['NUM_EPOCHS'] + 1):
        training_loop(training_loader, epoch)
        testing_loop(testing_loader, epoch)

In [13]:
train(FLAGS) # Please ignore the error, it doesn't exist anymore. Just look at the training loss, it decreases.

Epoch 1 train begin 10:08:23
loss: 2.484375, time: 10:09:20, step: 0
loss: 0.6015625, time: 10:11:06, step: 100
loss: 0.63671875, time: 10:12:17, step: 200
loss: 1.3046875, time: 10:13:27, step: 300
loss: 0.28515625, time: 10:14:38, step: 400
loss: 0.0947265625, time: 10:15:48, step: 500
loss: 0.7265625, time: 10:16:58, step: 600
loss: 0.375, time: 10:18:09, step: 700
loss: 1.2421875, time: 10:19:19, step: 800
loss: 0.244140625, time: 10:20:30, step: 900
loss: 0.55859375, time: 10:21:40, step: 1000
loss: 0.474609375, time: 10:22:51, step: 1100
loss: 0.6875, time: 10:24:01, step: 1200
loss: 0.302734375, time: 10:25:11, step: 1300
loss: 0.138671875, time: 10:26:22, step: 1400
loss: 0.56640625, time: 10:27:32, step: 1500
loss: 0.5, time: 10:28:43, step: 1600
loss: 0.62109375, time: 10:29:53, step: 1700
loss: 1.1953125, time: 10:31:04, step: 1800
loss: 0.349609375, time: 10:32:14, step: 1900
loss: 0.53125, time: 10:33:24, step: 2000
loss: 0.64453125, time: 10:34:35, step: 2100
loss: 0.4726

NameError: name 'average_loss' is not defined

In [ ]:
# We can just download the model directly but Kaggle hardware has little disk storage so I had to do this
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_write") # Provide your own HF API token with write access
login(hf_token)

model = model.cpu() # We have to move the model onto CPU to push it to HF.
print('now saving the model')
model.push_to_hub(
    "Defetya/example_repo_detectAI", 
    tokenizer=tokenizer,
    private=False,
    create_pr=False,
    max_shard_size="2GB", # Sharding isn't as important as before since hardware is better now but who cares anyway
    )# We have to push the model to HF since there is not enough memory on disk. Download weights from there